In [8]:
with open('21.txt') as f:
    codes = f.read().splitlines()

KEYPAD = tuple('''
789
456
123
.0A
'''.strip().splitlines())

ARROWS = tuple('''
.^A
<v>
'''.strip().splitlines())

# Part 1: Arrow decipherment

In [41]:
from functools import cache
from queue import Queue

@cache
def find(lines: tuple[str], match_char: str):
    for y, line in enumerate(lines):
        for x, char in enumerate(line):
            if char == match_char:
                return (x, y)

Coord = tuple[int, int, str]

def get_directions(x, y, ex, ey):
    if x < ex: yield '>'
    if x > ex: yield '<'
    if y < ey: yield 'v'
    if y > ey: yield '^'

def dijkstra(grid: tuple[str], start: str, end: str, N: int):
    start, end = find(grid, start), find(grid, end)

    start = (*start, 'A')

    UNSEEN_COST = 1e12
    frontier = Queue[Coord]()
    frontier.put(start)

    costs: dict[Coord, int] = {}
    costs[start] = 0
    prevs: dict[Coord, Coord] = {}
    prevs[start] = None

    while not frontier.empty():
        x, y, d0 = frontier.get()
        if (x, y) == end:
            break
        for d in get_directions(x, y, *end):
            sx, sy = x, y
            match d:
                case '<': sx -= 1
                case '>': sx += 1
                case '^': sy -= 1
                case 'v': sy += 1

            move_cost = 1
            if N > 0:
                move_cost = d_cost(ARROWS, d0, d, N-1)
                if (sx, sy) == end:
                    move_cost += d_cost(ARROWS, d, 'A', N-1)

            cost = costs[x, y, d0] + move_cost
            costs[sx, sy, d] = cost
            prevs[sx, sy, d] = (x, y, d0)
            frontier.put((sx, sy, d))

    return costs, prevs

@cache
def d_cost(grid: tuple[str], start: str, end: str, N: int):
    costs, _ = dijkstra(grid, start, end, N)
    end = find(grid, end)
    for d in '<>^v':
        if (*end, d) in costs:
            return costs[*end, d]
    

In [47]:
dijkstra(KEYPAD, '0', '3', 3)

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [7]:
def find(lines: list[str], match_char: str):
    for y, line in enumerate(lines):
        for x, char in enumerate(line):
            if char == match_char:
                return (x, y)

def encode(grid: list[str], code: str):
    x, y = find(grid, 'A')
    ax, ay = find(grid, '.')
    for char in code:
        sx, sy = find(grid, char)
        dx, dy = sx-x, sy-y
        vert = ('v' if dy > 0 else '^') * abs(dy)
        hori = ('>' if dx > 0 else '<') * abs(dx)
        if dx > 0:
            yield from vert
            yield from hori
        else:
            yield from hori
            yield from vert
        yield 'A'
        x, y = sx, sy

def decode(grid: list[str], code: str):
    x, y = find(grid, 'A')
    for char in code:
        match char:
            case '^': y -= 1
            case 'v': y += 1
            case '<': x -= 1
            case '>': x += 1
            case 'A': yield grid[y][x]

score = 0
for c0 in codes:
    c3 = ''.join(
        encode(ARROWS, encode(ARROWS, encode(KEYPAD, c0))))
    assert c0 == ''.join(
        decode(KEYPAD, decode(ARROWS, decode(ARROWS, c3)))
    )
    score += len(c3) * int(c0[:-1])
    print(f'{c0}: ({len(c3):>2}) {c3}')
score - 126384

029A: (68) <<vAA>A^>A<Av>AA^A<<vA^>>AvA^A<<vA^>>AA<vA>A^A<A>A<<vA>A^>AAA<Av>A^A
980A: (60) <<vA^>>AAAvA^A<<vAA>A^>A<Av>AA^A<<vA>A^>AAA<Av>A^A<vA^>A<A>A
179A: (64) <<vAA>A^>AA<Av>A^AvA^A<<vA^>>AAvA^A<vA^>AA<A>A<<vA>A^>AAA<Av>A^A
456A: (60) <<vAA>A^>AA<Av>A^AAvA^A<vA^>A<A>A<vA^>A<A>A<<vA>A^>AA<Av>A^A
379A: (64) <<vA^>>AvA^A<<vAA>A^>AA<Av>A^AAvA^A<vA^>AA<A>A<<vA>A^>AAA<Av>A^A


-2540

# Scratchpad

The order of arrows seems like it wouldn't matter, but after two generations it does:

In [3]:
for segment in ('<^', '<v', '>^', '>v'):
    for c0 in segment+'A', segment[::-1]+'A':
        (c1 := ''.join(encode(ARROWS, c0)))
        (c2 := ''.join(encode(ARROWS, c1)))
        print(c0, len(c2))
    print()

<^A 21
^<A 25

<vA 21
v<A 25

>^A 19
^>A 19

>vA 21
v>A 17



The most efficient path is to go horizontal first – unless `dx>0` and `dy>0`.